## Example workflow to register multi-view light sheet data

Notes:
- install napari-stitcher for visualization functionality
- registration: so far only translation registration is performed
- fusion: only vanilla linear blending currently supported
- generally
  - this is a first hacky workflow that will change in API and become simplified
  - documentation will follow

In [12]:
# imports

import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics, tempfile

from multiview_stitcher import msi_utils, spatial_image_utils

%matplotlib notebook

# the package napari-stitcher is required for visualization with napari
VISUALIZE_USING_NAPARI = False

if VISUALIZE_USING_NAPARI:
    import napari
    from napari_stitcher import viewer_utils

In [13]:
# Start a dask cluster

from distributed import Client, LocalCluster

lc = LocalCluster(n_workers=1, threads_per_worker=None)
client = Client(lc)
client


/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56828 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:56828/status,
Dashboard: http://127.0.0.1:56828/status,Workers: 1
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56829,Workers: 1
Dashboard: http://127.0.0.1:56828/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:56836,Total threads: 8
Dashboard: http://127.0.0.1:56837/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:56832,


## Specify input data

In [14]:
base_dir = '../image-datasets/multi-view/old_mDSLM_classical_4_angles_10x_0.3NA_detection'
filenames = [(os.path.join(base_dir, f)) for f in os.listdir(base_dir) if f.endswith('.tif')]

# sort angles
filenames = [Path(fn) for fn in sorted(filenames)]
print('Files:')
print('\n'.join([fn.name for fn in filenames]))


Files:
MGolden2022A-DS0031TP0001DR0001CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0002CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0003CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0004CH0001PL(ZS).tif


## Convert input data to OME-Zarr

In [15]:
from multiview_stitcher import io

overwrite = False

msims = []
for filename in tqdm(filenames):
    store_path = filename.with_suffix('.zarr')
    if not os.path.exists(store_path) or overwrite:
        sim = io.read_tiff_into_spatial_xarray(
                        filename,
                        scale={'z': 2.58, 'y': 0.645, 'x': 0.645})
        msim = msi_utils.get_msim_from_sim(sim, scale_factors=None) # choose scale factors automatically
        msim.to_zarr(store_path)
    msim = msi_utils.multiscale_spatial_image_from_zarr(Path(store_path))
    msims.append(msim)

# alternative
# msims = []
# for filename in tqdm(filenames):
#     msim = msi_utils.get_store_decorator(
#         filename.with_suffix('.zarr'),
#         store_overwrite=False)(
#             msi_utils.get_msim_from_sim)(
#                 io.read_tiff_into_spatial_xarray(
#                     filename,
#                     scale={'z': 2.58, 'y': 0.645, 'x': 0.645}
#                 ))
#     msims.append(msim)


100%|██████████| 4/4 [00:00<00:00,  4.60it/s]


In [16]:
print(msims[0])

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       multiscaleSpatialImageVersion:  1
│       multiscales:                    [{'@type': 'ngff:Image', 'axes': [{'name'...
├── DataTree('scale0')
│       Dimensions:          (t: 1, x_in: 4, x_out: 4, c: 1, z: 179, y: 1040, x: 1392)
│       Coordinates:
│         * c                (c) int64 0
│         * t                (t) int64 0
│         * x                (x) float64 0.0 0.645 1.29 1.935 ... 895.9 896.6 897.2
│         * y                (y) float64 0.0 0.645 1.29 1.935 ... 668.9 669.5 670.2
│         * z                (z) float64 0.0 2.58 5.16 7.74 ... 451.5 454.1 456.7 459.2
│       Dimensions without coordinates: x_in, x_out
│       Data variables:
│           affine_metadata  (t, x_in, x_out) float64 1.0 0.0 0.0 0.0 ... 0.0 0.0 1.0
│           image            (t, c, z, y, x) uint16 dask.array<chunksize=(1, 1, 179, 256, 256), meta=np.ndarray>
├── DataTree('scale1')
│   

## Set estimate of initial transformations

In [17]:
import transformations as tf

for imsim, msim in enumerate(msims):

    # 90 degree rotation around x axis
    affine = tf.rotation_matrix(
        -np.pi/2 * imsim,
        point=spatial_image_utils.get_center_of_sim(msims[imsim]['scale0/image'], transform_key=None),
        direction=[0,0,1],
        )
    
    msi_utils.set_affine_transform(
        msim,
        affine[None], # one tp
        transform_key='affine_manual',
    )


In [23]:
print(msims[0]['scale0/affine_manual'])

<xarray.DataArray 'affine_manual' (t: 1, x_in: 4, x_out: 4)>
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]])
Coordinates:
  * t        (t) int64 0
Dimensions without coordinates: x_in, x_out


In [22]:
print(msims[0]['scale0/image'])

<xarray.DataArray 'image' (t: 1, c: 1, z: 179, y: 1040, x: 1392)>
dask.array<open_dataset-c17dbf5006081e3ae049471d3b3ca9e6image, shape=(1, 1, 179, 1040, 1392), dtype=uint16, chunksize=(1, 1, 179, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 0
  * t        (t) int64 0
  * x        (x) float64 0.0 0.645 1.29 1.935 2.58 ... 895.3 895.9 896.6 897.2
  * y        (y) float64 0.0 0.645 1.29 1.935 2.58 ... 668.2 668.9 669.5 670.2
  * z        (z) float64 0.0 2.58 5.16 7.74 10.32 ... 451.5 454.1 456.7 459.2


### Visualize pre-registered views

In [ ]:
if VISUALIZE_USING_NAPARI:

    viewer = napari.Viewer(ndisplay=3)
    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_manual', n_colors=4, contrast_limits=[0, 1000])
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)
    viewer.reset_view()
    

### Register views

In [ ]:
# import skimage.registration
from multiview_stitcher import registration

with dask.diagnostics.ProgressBar():

    params = registration.register(
        # [msi_utils.get_sim_from_msim(msim) for msim in msims],
        msims,
        registration_binning={'z': 2, 'y': 8, 'x': 8},
        reg_channel_index=0,
        transform_key='affine_manual',
    )
    
for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_manual')

### Visualize registration

In [ ]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_manual', n_colors=4,
        name_prefix='pre-registered view',
        contrast_limits=[0, 1000],
        )
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='registered view',
        contrast_limits=[0, 1000],
        )
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

### Fuse views (linear blending)

In [ ]:
from multiview_stitcher import fusion
import dask.array as da

sims = [msi_utils.get_sim_from_msim(msim) for msim in msims]

tmpdir = tempfile.TemporaryDirectory()

import importlib
fusion = importlib.reload(fusion)

fused = fusion.fuse(
    sims[:],
    transform_key='affine_registered',
    output_spacing={dim: 10 for dim in ['z', 'y', 'x']},
    output_chunksize=128,
    )

print('Fusing views...')
with dask.diagnostics.ProgressBar():

    fused.data = da.to_zarr(
        fused.data,
        os.path.join(tmpdir.name, 'fused_sim.zarr'),
        overwrite=True, return_stored=True, compute=True)

print('Creating multiscale output OME-Zarr...')
with dask.diagnostics.ProgressBar():

    mfused = msi_utils.get_msim_from_sim(fused, scale_factors=None)

    fused_path = os.path.join(tmpdir.name, 'fused.zarr')
    mfused.to_zarr(fused_path)
    
mfused = msi_utils.multiscale_spatial_image_from_zarr(fused_path)

### Visualize fusion in napari

In [ ]:
if VISUALIZE_USING_NAPARI:

    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='registered view',
        contrast_limits=[0, 1000]
        )

    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(
        viewer, lds, do_link_layers=False)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused,
        transform_key='affine_registered',
        name_prefix='fused',
        contrast_limits=[0, 1000])

    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)

In [ ]:
# stream presaved fused image to tif

from multiview_stitcher import io

with dask.diagnostics.ProgressBar():
    io.save_sim_as_tif('fused.tif', msi_utils.get_sim_from_msim(mfused))